In [ ]:
# Module Importations

In [ ]:
# Custom Modules

In [ ]:
# Constants

In [ ]:
# Load Dataset

# Identify Suspect Faulty Panels

An assumption is that panels will have zero output if they are in a fault condition (as opposed to a lower output). This assumption is supported by a working principle of inverters that they will stop producing an output if the input voltage falls too low.

Panels with larger numbers of zero-output instances may be faulty.

This can be visualised with a cut-off region.